In [ ]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read data into pandas dataframe
kick_data = pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter002.csv")
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter003.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter004.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter005.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter006.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter007.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter008.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter009.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter010.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter011.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter012.csv"))

# Isolate relevant attributes
kick_data = kick_data.loc[:, ['backers_count','blurb','category','country', 'goal', 
                              'staff_pick', 'state', 'usd_pledged']]

# remove live entries as their final state is unknown
kick_data = kick_data[kick_data.state != 'live']

# remove rows with missing values
kick_data = kick_data.dropna()

# reset index for model-building and to remove duplicate values
kick_data = kick_data.reset_index(drop=True)

# initial count will return 40169 for most categories, 40167 for blurb column
kick_data.count()

kick_data.tail()

In [ ]:
# Return entries per category with a count of less than a certain int, set to zero for all
kick_data['category'].value_counts()[kick_data['category'].value_counts()>0]

In [ ]:
# Return count of unique values
kick_data['category'].nunique()

In [ ]:
# Count entries per value in country column
kick_data['country'].value_counts()

In [ ]:
# Count entries per value in staff_pick column
kick_data['staff_pick'].value_counts()

In [ ]:
# Count entries per value in state column
kick_data['state'].value_counts()

In [ ]:
# Construct seaborn jointplot
sns.jointplot(data=kick_data, x='backers_count', y='usd_pledged', color='b')
plt.show()

In [ ]:
# state values changed to integers for correlation analysis
kick_data.loc[:,"state"] = kick_data.loc[:,"state"].apply(lambda x: 1 if x == 'successful' else 0)
# check that state values are int
kick_data.info() 

In [ ]:
# Construct seaborn heatmap to get correlations between attributes
sns.heatmap(kick_data.corr(), annot=True, cmap='PRGn')
plt.show()

In [ ]:
# Construct seaborn boxplot to see how country affects how much is pledged
sns.boxplot(x="country", y="usd_pledged", data=kick_data)
plt.show()

In [ ]:
# Count entries per country based on staff_pick
kick_pt = kick_data.pivot_table(index='country', columns='staff_pick', aggfunc='size')
sns.heatmap(kick_pt, annot=True, fmt=".0f", cmap = "PRGn")
plt.show()

In [ ]:
# Construct heatmap to show median value of usd_pledged based on country and staff pick
kick_pt = kick_data.pivot_table(index='country', columns='staff_pick', values='usd_pledged', aggfunc=np.median)
sns.heatmap(kick_pt, annot=True, fmt=".2f", cmap = "PRGn")
plt.show()

In [ ]:
# kNN model

z = 32000

# feature variables
x_train = kick_data.loc[:z, ['backers_count','goal', 'staff_pick', 'usd_pledged']]
x_test = kick_data.loc[z:, ['backers_count','goal', 'staff_pick', 'usd_pledged']]
# response variable
y_train = kick_data.loc[:z, 'state'] 
y_test = kick_data.loc[z:, 'state']

from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
import math as m

# find best k
best_k = 0
best_accuracy = 0

for i in range (1, m.floor(len(x_test)/50)):
    # training model on training set
    knn = KNeighborsClassifier(n_neighbors=i) 
    knn.fit(x_train, y_train) 
    # making predictions on testing set 
    y_pred = knn.predict(x_test) 
    a = metrics.accuracy_score(y_test, y_pred)
    if a > best_accuracy:
        best_k = i
        best_accuracy = a
print(best_k,best_accuracy)

In [ ]:
# usd_pledged as y, backers, country, staff_pick
from sklearn import linear_model

x_train=input_variables_values_training_datasets
y_train=target_variables_values_training_datasets
x_test=input_variables_values_test_datasets
# Create linear regression object
linear = linear_model.LinearRegression()
# Train the model using the training sets and check score
linear.fit(x_train, y_train)
linear.score(x_train, y_train)
#Equation coefficient and Intercept
print('Coefficient: \n', linear.coef_)
print('Intercept: \n', linear.intercept_)
#Predict Output
predicted= linear.predict(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create logistic regression object
model = LogisticRegression()
# Train the model using the training sets and check score
model.fit(X, y)
model.score(X, y)
#Equation coefficient and Intercept
print('Coefficient: \n', model.coef_)
print('Intercept: \n', model.intercept_)
#Predict Output
predicted= model.predict(x_test)

In [ ]:
from sklearn import tree
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create tree object 
model = tree.DecisionTreeClassifier(criterion='gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score
model.fit(X, y)
model.score(X, y)
#Predict Output
predicted= model.predict(x_test)